#**Lab 4 and 5: Neural Machine Translation (Extra Guide)**

This week and the next, we will build a neural machine translation model based on the sequence-to-sequence (seq2seq) models proposed by Sutskever et al., 2014 and Cho et al., 2014. The seq2seq model is widely used in Machine Translation systems such as Google’s neural machine translation system (GNMT) (Wu et al., 2016).

The folder **nmt_lab_files** has been provided for you. This folder contains 3 files:
1. **data.30.vi** - a file where each line contains a Vietnamese sentence to be translated (i.e. the source sentences)
2. **data.30.en** - a file where each line contains an English sentence corresponding to the Vietnamese sentence in the same line position. (i.e. the target sentences)
3. **nmt_model_keras.py** - incomplete code for this lab.

The pdf file provided contains an explanation of the code file and a guide on how to complete the code (by doing 3 tasks). Read the pdf file and complete the code as instructed.

##**LanguageDict**

LanguageDict is a class for creating language dict objects.

## **The load_dataset() Method**

This helper method reads from the source and target files to load max_num_examples sentences, split those sentences into train, development and test sets, and return relevant data.
 
As an example of the ouput returned by this code, let's assume we are translating the sentence 'I like rabbits' from English to English (this of course is never the case), such that the tokenised and case-normalised source sentence list and target sentence list are as follows:


```
# In Vietnamese this would actually be [['tôi', 'thích', 'thỏ']]. 
# We will use English to English here using the following code.
source_words = [['i', 'like', 'rabbits']] 
target_words = [['i', 'like', 'rabbits']]
```
The word2ids for the source and target language dictionaries look as follows:
```
source_dict.word2ids = {'<PAD>': 0, '<UNK>': 1, 'i': 2, 'like': 3, 'rabbits':4}

# end and start tokens are added to the target words
target_dict.word2ids = {'<PAD>': 0, '<UNK>': 1, '<start>': 2, 'i': 3, 'like': 4, 'rabbits':5, '<end>':6}

```
Let's also assume that we are training and testing on this dataset of one sentence.
The **source words** for train/dev/test will be given as follows:
```
# [batch_size X max_sent_length]
source_words_train = [[2,3,4]] # corresponding to ['i', 'like', 'rabbits']
source_words_dev = [[2,3,4]]  # corresponding to ['i', 'like', 'rabbits']
source_words_test = [[2,3,4]] # corresponding to ['i', 'like', 'rabbits']
```

The **target words** for the train data will be given as follows (dev/test do not need target words as the model will generate those):
```
target_words_train = [[2,3,4,5]] # corresponding to ['<start>', 'i', 'like', 'rabbits']
```

The **target words labels** for each word will be the next word. The target word labels for train/dev/test data will be given as follows
```
target_words_train_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
target_words_dev_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
target_words_test_labels = [[3,4,5,6]] # corresponding to ['i', 'like', 'rabbits', '<end>']
```
The dimensions for the train target words labels would be expanded to have the following dimentionality:
```
# [batch_size X max_sent_length array X 1]
[[3], [4], [5], [6]]
```






##**Neural Translation Model (NMT)**

For NMT, the network (a system of connected layers/models) used for training differs slightly from the network used for inference. Both use the encoder-decoder architecture. 




###**Training mode**

**Encoder**

Given:
- `source_words`: a `batch_size(num_sents) x max_sentence_length` array representing the source words. In our mini example, this would be the Vietnamese equivalent of `['i', 'like', 'rabbits']`, i.e. `[['tôi', 'thích', 'thỏ']]`.

The following steps comprise the encoder network:

1. Transform `source_words` into `source_words_embeddings` using a randomly initialized embedding lookup. `source_words_embeddings` is thus an array with the shape `batch_size(num_sents) x max_sentence_length x embedding_dim`.
2. Apply embedding dropout with `embedding_dropout_rate`.
3. Use a single `LSTM` with the `hidden_size` units to learn a representation for the source words i.e. to encode the input. 

    (a.) The hidden and cell states for this `LSTM` are initialized to zeros (i.e. we leave the `initial_state = None` default as is).

    (b.) We save the `encoder_outputs` (the sequence not just the last state); and the encoder (hidden and cell) states. 

This way, the model encodes a representation for the source words. Task 1 guides you to complete the encoder part of the training model.


**Decoder (No Attention)**

Given:
- `target_words`: a `batch_size(i.e. num_sents in batch) x max_sentence_length` array representing the target words. This is a time shifted translation of the source words with an added (prepended) `<START>` token `['<start>', 'i', 'like', 'rabbits']`.

The decoding is done in the following steps:

1. Transform `target_words` into `target_words_embeddings` using a randomly initialized embedding lookup. `target_words_embeddings` is thus an array with the shape `batch_size x max_sentence_length x embedding_dim`.

2. Apply embedding dropout of `embedding_dropout_rate`.

3. Use a single `LSTM` with `hidden_size` units to learn a representation for the target words. The context is given to this model by using the encoder states to initialise the decoder LSTM. For example, the encoder state for `'thỏ'` (last word in the input sequence, its hidden representation summarises the sentence) is used to learn the representation for the `'<start>'` token.

4. For each token representation, we use a dense layer to output a `target_vocab_size` vector of probabilities to be the next word following the represented token. The output `decoder_outputs_train` is thus an array  with the shape `batch_size x max_sent_length + 1 x target_vocab_size`.


###**Inference Mode**

**Encoder**

The inference time encoding follows the same steps as the training time encoding.


**Decoder (No attention)**

During training time, we passed a `batch_size(num_sents) x max_sentence_length` array representing the target words into the decoder LSTM. The `decoder_lstm` represents the given target sentence using the context from the encoder LSTM (representation for the source sentence).  

At test time, several things are different:

1. We no longer have access to a complete translation of the source sentence (recall that no `target_words` arrays exist for dev and test sets). Rather we initialise the target words array as follows:

    Each expected target sentence contains only a single token index, the index of the `'<start>'` token. So, the target_word_dev/test is a `batch_size x 1` array (see the nmt.eval() function).

2. This `batch_size x 1` array is fed to the trained `decoder_lstm` and the predicted array is a `batch_size x 1 x target_vocab_size` such that taking the argmax of this array across the dimension 2 will give the most probable next word. 

For example, at time_step 0 (first time step) the `step_target_words` is given. It is a `batch_size x 1` array containing the `'<start>'` token. The next word prediction of the decoder is for each sentence (in the batch) the first actual word. 


At the first time step, the `decoder_lstm` still uses the `encoder_states` as its initial states. At subsequent time steps, it uses its own states from the previous time steps. We hence loop over time steps to generate a new word at a time.





In [ ]:
PATH_TO_FOLDER = "/home/jovyan/ECS7001p/anotherone/nmt_lab_files"

In [ ]:
SOURCE_PATH = PATH_TO_FOLDER + '/data.30.vi'
TARGET_PATH = PATH_TO_FOLDER + '/data.30.en'

Let's install the Sacrebleu (https://github.com/mjpost/sacrebleu) package for BLEU computation.

In [ ]:
import os
import sys
sys.path.insert(0,"/home/jovyan/ECS7001p/anotherone/nmt_lab_files")

In [ ]:
sys.path

['/home/jovyan/ECS7001p/anotherone/nmt_lab_files',
 '/home/jovyan/ECS7001p/anotherone',
 '/usr/local/etc/jupyter',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages']

In [ ]:
!pip install sacrebleu
!pip install keras


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install tf


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import nmt_model_keras as nmt

##**Training Without Attention**

If you have completed Tasks 1 and 2, you are ready to train the NMT model without attention.

Run the following cells to train the model for 10 epochs. The model summary is also shown below.

If you're using a GPU, training will be no more than 10 minutes and you will get the test BLEU score between 5 and 6. 

In [ ]:
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=False)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
Not using any device.
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder


2023-03-08 13:30:11.023466: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 13:30:11.729426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2631 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:e3:00.0, compute capability: 8.6


						 Train Model Summary.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    203400      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 100)    250600      ['input_2[0][0]']                
                                                                  

/opt/conda/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


None
Starting training epoch 1/10
  1/240 [..............................] - ETA: 19:26 - loss: 3.7441 - accuracy: 0.0000e+00

...

Model BLEU score: 5.72
Time used for evaluate on dev set: 0 m 10 s
Starting training epoch 10/10
240/240 [==============================] - 30s 123ms/step - loss: 1.3927 - accuracy: 0.3992
Time used for epoch 10: 0 m 29 s
Evaluating on dev set after epoch 10/10:


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


Model BLEU score: 5.67
Time used for evaluate on dev set: 0 m 10 s
Training finished!
Time used for training: 6 m 46 s
Evaluating on test set:


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


Model BLEU score: 6.14
Time used for evaluate on test set: 0 m 10 s


##**Training and Decoding with Attention**

The inputs to the attention layer are encoder and decoder outputs. The attention mechanism:
1. Computes a score (Luong's dot product attention score) for each source word
2. Weights the encoder representations using these scores.
3. Concatenates the weighted encoder representation with the decoder ouput.
This new decoder output will now be the input to the `decoder_dense` layer. 

Step-by-step details for Task 3 are in the pdf file. Once you have completed this Task, you are ready to train with attention. Training time will be no more than 10 minutes using a GPU and you should get a test BLEU score of around 15.

In [ ]:
nmt.main(SOURCE_PATH, TARGET_PATH, use_attention=True)

loading dictionaries
read 24000/3000/3000 train/dev/test batches
Not using any device.
number of tokens in source: 2034, number of tokens in target:2506
Task 1(a): Creating the embedding lookups...

Task 1(b): Looking up source and target words...

Task 1(c): Creating an encoder


2023-03-08 13:51:19.600778: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 13:51:20.351143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2631 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:e3:00.0, compute capability: 8.6


						 Train Model Summary.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 100)    203400      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 dropout (Dropout)              (None, None, 100)    0           ['embedding[0][0]']              
                                                                  

/opt/conda/lib/python3.10/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


 dropout_1 (Dropout)            (None, None, 100)    0           ['embedding_1[0][0]']            
                                                                                                  
 input_3 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 200)]        0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, None, 200)]  0           []                               
                                                                                                  
 lstm_1 (LSTM)                  [(None, None, 200),  240800      ['dropout_1[0][0]',              
                                 (None, 200),                     'input_3[0][0]',                
          

...

Model BLEU score: 15.11
Time used for evaluate on dev set: 0 m 10 s
Starting training epoch 10/10
240/240 [==============================] - 30s 127ms/step - loss: 0.9070 - accuracy: 0.5531
Time used for epoch 10: 0 m 30 s
Evaluating on dev set after epoch 10/10:


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


Model BLEU score: 14.65
Time used for evaluate on dev set: 0 m 10 s
Training finished!
Time used for training: 6 m 58 s
Evaluating on test set:


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


Model BLEU score: 14.98
Time used for evaluate on test set: 0 m 10 s
